In [31]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc,count,split,col,max,min,countDistinct

In [2]:
spark=SparkSession.builder.appName("First App").getOrCreate()

In [19]:
path='D:/Expedia/HandsOn/Spark/input/deliveries.csv'
path1='D:/Expedia/HandsOn/Spark/input/matches.csv'
df=spark.read.option("header","true").option("inferShema","true").csv(path)
match_df=spark.read.option("header","true").option("inferShema","true").csv(path1)

In [4]:
df.show(3)

+--------+------+-------------------+--------------------+----+----+---------+-----------+--------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------+
|match_id|inning|       batting_team|        bowling_team|over|ball|  batsman|non_striker|  bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|player_dismissed|dismissal_kind|fielder|
+--------+------+-------------------+--------------------+----+----+---------+-----------+--------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------+
|       1|     1|Sunrisers Hyderabad|Royal Challengers...|   1|   1|DA Warner|   S Dhawan|TS Mills|            0|        0|       0|          0|          0|           0|           0|         0|         0|            null|          null|   null|
|       1|     1|Sun

In [5]:
df.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- inning: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: string (nullable = true)
 |-- ball: string (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- is_super_over: string (nullable = true)
 |-- wide_runs: string (nullable = true)
 |-- bye_runs: string (nullable = true)
 |-- legbye_runs: string (nullable = true)
 |-- noball_runs: string (nullable = true)
 |-- penalty_runs: string (nullable = true)
 |-- batsman_runs: string (nullable = true)
 |-- extra_runs: string (nullable = true)
 |-- total_runs: string (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



In [7]:
df.withColumn('inning',col('inning').cast('int'))

DataFrame[match_id: string, inning: int, batting_team: string, bowling_team: string, over: string, ball: string, batsman: string, non_striker: string, bowler: string, is_super_over: string, wide_runs: string, bye_runs: string, legbye_runs: string, noball_runs: string, penalty_runs: string, batsman_runs: string, extra_runs: string, total_runs: string, player_dismissed: string, dismissal_kind: string, fielder: string]

# Converting Numerical columns' datatype from string to int

In [13]:
columns_to_cast=['match_id','inning','over','ball','is_super_over','wide_runs','bye_runs','legbye_runs','noball_runs','penalty_runs','batsman_runs','extra_runs','total_runs']

In [14]:
for column in  columns_to_cast:
    df=df.withColumn(column,col(column).cast('int'))

# Total number of matches palyed

In [16]:

df.select('match_id').count()

150460

In [21]:
match_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- result: string (nullable = true)
 |-- dl_applied: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_runs: string (nullable = true)
 |-- win_by_wickets: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- umpire3: string (nullable = true)



In [23]:
match_df.select('season').distinct().show()

+------+
|season|
+------+
|  2016|
|  2012|
|  2017|
|  2014|
|  2013|
|  2009|
|  2011|
|  2008|
|  2015|
|  2010|
+------+



In [24]:
match_df=match_df.withColumn('season',col('season').cast('int'))

In [26]:
match_df.groupBy('season').agg(count('id').alias('match_count')).sort('season').show()

+------+-----------+
|season|match_count|
+------+-----------+
|  2008|         58|
|  2009|         57|
|  2010|         60|
|  2011|         73|
|  2012|         74|
|  2013|         76|
|  2014|         60|
|  2015|         59|
|  2016|         60|
|  2017|         59|
+------+-----------+



# City having More than one statdium for IPL

In [42]:
match_df.groupBy('city').agg(countDistinct('venue').alias('venue_count')).filter(col('venue_count')>1).show()

+----------+-----------+
|      city|venue_count|
+----------+-----------+
|    Mumbai|          3|
|      Pune|          2|
|Chandigarh|          2|
+----------+-----------+



# Top 5 Stadium with maximum match being played

In [41]:
city_venue_map=match_df.select('city','venue').distinct()

In [43]:
match_df.groupBy('city').agg(count('id').alias('match_count')).sort(desc('match_count')).show()

+-------------+-----------+
|         city|match_count|
+-------------+-----------+
|       Mumbai|         85|
|    Bangalore|         66|
|      Kolkata|         61|
|        Delhi|         60|
|    Hyderabad|         49|
|      Chennai|         48|
|   Chandigarh|         46|
|       Jaipur|         33|
|         Pune|         32|
|       Durban|         15|
|    Ahmedabad|         12|
|    Centurion|         12|
|Visakhapatnam|         11|
|       Rajkot|         10|
|   Dharamsala|          9|
| Johannesburg|          8|
|         null|          7|
|       Ranchi|          7|
|    Cape Town|          7|
|    Abu Dhabi|          7|
+-------------+-----------+
only showing top 20 rows



In [40]:
venue_df=match_df.groupBy('venue').agg(count('id').alias('match_count'))
venue_df.join(city_venue_map,on='venue').sort(desc('match_count')).show(5,False)

+-----------------------------------------+-----------+---------+
|venue                                    |match_count|city     |
+-----------------------------------------+-----------+---------+
|M Chinnaswamy Stadium                    |66         |Bangalore|
|Eden Gardens                             |61         |Kolkata  |
|Feroz Shah Kotla                         |60         |Delhi    |
|Wankhede Stadium                         |57         |Mumbai   |
|Rajiv Gandhi International Stadium, Uppal|49         |Hyderabad|
+-----------------------------------------+-----------+---------+
only showing top 5 rows



# Top 3 match winning team

In [44]:
match_df.groupBy('winner').agg(count('id').alias('winning_match_count')).sort(desc('winning_match_count')).show(3,False)

+---------------------+-------------------+
|winner               |winning_match_count|
+---------------------+-------------------+
|Mumbai Indians       |92                 |
|Chennai Super Kings  |79                 |
|Kolkata Knight Riders|77                 |
+---------------------+-------------------+
only showing top 3 rows



# Number matches tie and No Result

In [47]:
match_df.select('result').distinct().show()

+---------+
|   result|
+---------+
|   normal|
|      tie|
|no result|
+---------+



In [58]:
match_df.groupBy('result').count().filter("result='tie' or result='no result'").show()

+---------+-----+
|   result|count|
+---------+-----+
|      tie|    7|
|no result|    3|
+---------+-----+



In [59]:
print(match_df.columns)

['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner', 'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs', 'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2', 'umpire3']


In [61]:
match_df.select('winner','win_by_runs').agg(max('win_by_runs')).show()

+----------------+
|max(win_by_runs)|
+----------------+
|              98|
+----------------+

